In [1]:
#include <vector>
#include <iostream>
#include <string>
#include <algorithm>

In [2]:
template <typename T>
std::ostream & operator << (std::ostream & os, const std::vector<T> & vec)
{
    for(auto elem : vec)
    {
        os<<elem<< " ";
    }
    return os;
}

## 1 create a vector from an existing vector: call copy constructor

In [3]:
std::vector<std::string> a_vector = {"a","ccc","ddd","e"};

In [4]:
std::vector<std::string> another(a_vector);

In [5]:
std::cout << another << std::endl;

a ccc ddd e 


## 2 create a vector from last elements of an existing vector

In [6]:
std::int64_t n = 3;

In [7]:
std::cout << std::min(int(n),int(a_vector.size())); 

3

In [8]:
std::vector<std::string> last_n_vector(a_vector.end()-std::min(int(n),int(a_vector.size())), a_vector.end());

In [9]:
std::cout << last_n_vector << std::endl;

ccc ddd e 


## 3 concat two vector

In [10]:
std::vector<int> x = { 1, 2, 7};
std::vector<int> y = { 4, 5 };
 
// 1. Copy constructor + vector::insert
 
std::vector<int> v(x);
v.insert(v.end(), y.begin(), y.end());

In [11]:
std::cout << v << std::endl;

1 2 7 4 5 
